In [690]:
import pandas as pd
import numpy as np

In [691]:
# pd.set_option('display.max_rows', None)

In [692]:
df = pd.read_csv('../data/raw/retail_store.csv')

In [693]:
sandbox = df.copy()

In [694]:
sandbox.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


In [695]:
sandbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


In [696]:
sandbox.isna().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [697]:
initial_count = len(sandbox)
initial_count

12575

In [698]:
sandbox.isnull().sum().sum()

np.int64(7229)

# Rename columns to title case with underscores #

In [699]:
sandbox.columns

Index(['Transaction ID', 'Customer ID', 'Category', 'Item', 'Price Per Unit',
       'Quantity', 'Total Spent', 'Payment Method', 'Location',
       'Transaction Date', 'Discount Applied'],
      dtype='object')

In [700]:
sandbox.columns.to_list()

['Transaction ID',
 'Customer ID',
 'Category',
 'Item',
 'Price Per Unit',
 'Quantity',
 'Total Spent',
 'Payment Method',
 'Location',
 'Transaction Date',
 'Discount Applied']

In [701]:
cols_to_rename = ['Transaction ID',
 'Customer ID',
 'Category',
 'Item',
 'Price Per Unit',
 'Quantity',
 'Total Spent',
 'Payment Method',
 'Location',
 'Transaction Date',
 'Discount Applied']

In [702]:
sandbox.rename(columns={ col: col.title().replace(' ', '_') for col in cols_to_rename}, inplace=True)

In [703]:
sandbox.head()

,Transaction_Id,Customer_Id,Category,Item,Price_Per_Unit,Quantity,Total_Spent,Payment_Method,Location,Transaction_Date,Discount_Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


# Standardize text columns #

In [704]:
string_cols = sandbox.select_dtypes(include=['object']).columns
string_cols

Index(['Transaction_Id', 'Customer_Id', 'Category', 'Item', 'Payment_Method',
       'Location', 'Transaction_Date', 'Discount_Applied'],
      dtype='object')

In [705]:
terms_to_cleam = ['ERROR', 'error', 'UNKNOWN', 'Unknown', 'NAN', 'nan', 'None', 'Na', '']

In [706]:
for col in string_cols:
    if col in sandbox.columns:
        sandbox[col] = (
            sandbox[col]
            .astype(str, errors='ignore')
            .str.lower()
            .str.replace(' ', '_')
            .replace(terms_to_cleam, np.nan)
            .str.strip())


In [707]:
sandbox.head()

,Transaction_Id,Customer_Id,Category,Item,Price_Per_Unit,Quantity,Total_Spent,Payment_Method,Location,Transaction_Date,Discount_Applied
0,txn_6867343,cust_09,patisserie,item_10_pat,18.5,10.0,185.0,digital_wallet,online,2024-04-08,true
1,txn_3731986,cust_22,milk_products,item_17_milk,29.0,9.0,261.0,digital_wallet,online,2023-07-23,true
2,txn_9303719,cust_02,butchers,item_12_but,21.5,2.0,43.0,credit_card,online,2022-10-05,false
3,txn_9458126,cust_06,beverages,item_16_bev,27.5,9.0,247.5,credit_card,online,2022-05-07,NaN
4,txn_4575373,cust_05,food,item_6_food,12.5,7.0,87.5,digital_wallet,online,2022-10-02,false


# Standardize numeric columns #

In [708]:
numeric_cols = sandbox.select_dtypes(include=('number')).columns
numeric_cols

Index(['Price_Per_Unit', 'Quantity', 'Total_Spent'], dtype='object')

In [709]:
for col in numeric_cols:
    if col in sandbox.columns:
        sandbox[col] = pd.to_numeric(sandbox[col], errors='coerce')

In [710]:
sandbox.tail()

,Transaction_Id,Customer_Id,Category,Item,Price_Per_Unit,Quantity,Total_Spent,Payment_Method,Location,Transaction_Date,Discount_Applied
12570,txn_9347481,cust_18,patisserie,item_23_pat,38.0,4.0,152.0,credit_card,in-store,2023-09-03,NaN
12571,txn_4009414,cust_03,beverages,item_2_bev,6.5,9.0,58.5,cash,online,2022-08-12,false
12572,txn_5306010,cust_11,butchers,item_7_but,14.0,10.0,140.0,cash,online,2024-08-24,NaN
12573,txn_5167298,cust_04,furniture,item_7_fur,14.0,6.0,84.0,cash,online,2023-12-30,true
12574,txn_2407494,cust_23,food,item_9_food,17.0,3.0,51.0,cash,online,2022-08-06,NaN


In [711]:
sandbox['Transaction_Date'] = pd.to_datetime(sandbox['Transaction_Date'], errors='coerce', infer_datetime_format=True)

C:\Users\Renato\AppData\Local\Temp\ipykernel_9152\2056610404.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  sandbox['Transaction_Date'] = pd.to_datetime(sandbox['Transaction_Date'], errors='coerce', infer_datetime_format=True)


# Analysis of the Price_Per_Unit column #

Upon analyzing the dataset, I observed that the Item and Price_Per_Unit columns could potentially be populated by mapping each item index to its corresponding unit price, given that each index is unique. Nevertheless, there are 609 missing entries in Item and over one thousand in Price_Per_Unit. Considering the presence of the Quantity and Total_Spent columns, it would be more logical to derive the Price_Per_Unit values from them first.

In [712]:
sandbox['Price_Per_Unit'] = np.where(
    sandbox['Price_Per_Unit'].isna() & (sandbox['Quantity'] > 0), 
    sandbox['Total_Spent'] / sandbox['Quantity'], 
    sandbox['Price_Per_Unit']
)

In [713]:
sandbox.isna().sum()

Transaction_Id         0
Customer_Id            0
Category               0
Item                1213
Price_Per_Unit         0
Quantity             604
Total_Spent          604
Payment_Method         0
Location               0
Transaction_Date       0
Discount_Applied    4199
dtype: int64

# Analysis of the Category column #

In [714]:
sandbox['Category'].sort_values().unique()

array(['beverages', 'butchers', 'computers_and_electric_accessories',
       'electric_household_essentials', 'food', 'furniture',
       'milk_products', 'patisserie'], dtype=object)

In [715]:
sandbox['Category'].value_counts()

Category
electric_household_essentials         1591
furniture                             1591
food                                  1588
milk_products                         1584
butchers                              1568
beverages                             1567
computers_and_electric_accessories    1558
patisserie                            1528
Name: count, dtype: int64

In [716]:
sandbox.groupby('Category')['Item'].unique()

Category
beverages                             [item_16_bev, nan, item_7_bev, item_25_bev, it...
butchers                              [item_12_but, item_22_but, item_3_but, item_20...
computers_and_electric_accessories    [item_16_cea, item_5_cea, item_15_cea, item_24...
electric_household_essentials         [item_13_ehe, item_4_ehe, item_21_ehe, item_10...
food                                  [item_6_food, item_1_food, item_2_food, item_1...
furniture                             [nan, item_16_fur, item_14_fur, item_25_fur, i...
milk_products                         [item_17_milk, nan, item_16_milk, item_19_milk...
patisserie                            [item_10_pat, nan, item_24_pat, item_17_pat, i...
Name: Item, dtype: object

In [717]:
sandbox.groupby(['Item', 'Category'])['Price_Per_Unit'].unique()

Item          Category                          
item_10_bev   beverages                             [18.5]
item_10_but   butchers                              [18.5]
item_10_cea   computers_and_electric_accessories    [18.5]
item_10_ehe   electric_household_essentials         [18.5]
item_10_food  food                                  [18.5]
                                                     ...  
item_9_ehe    electric_household_essentials         [17.0]
item_9_food   food                                  [17.0]
item_9_fur    furniture                             [17.0]
item_9_milk   milk_products                         [17.0]
item_9_pat    patisserie                            [17.0]
Name: Price_Per_Unit, Length: 200, dtype: object

Given that each item index holds a unique value. I will extract the corresponding data and build a mapping dictionary to address inaccurate or missing records in the Price_Per_Unit column.  

In [718]:
sandbox['Item'].str.split('_')

0         [item, 10, pat]
1        [item, 17, milk]
2         [item, 12, but]
3         [item, 16, bev]
4         [item, 6, food]
               ...       
12570     [item, 23, pat]
12571      [item, 2, bev]
12572      [item, 7, but]
12573      [item, 7, fur]
12574     [item, 9, food]
Name: Item, Length: 12575, dtype: object

In [719]:
sandbox['Item'].str.split('_').str[1]

0        10
1        17
2        12
3        16
4         6
         ..
12570    23
12571     2
12572     7
12573     7
12574     9
Name: Item, Length: 12575, dtype: object

In [720]:
sandbox['Item_Number'] = sandbox['Item'].str.split('_').str[1].astype(float)
sandbox['Item_Number']

0        10.0
1        17.0
2        12.0
3        16.0
4         6.0
         ... 
12570    23.0
12571     2.0
12572     7.0
12573     7.0
12574     9.0
Name: Item_Number, Length: 12575, dtype: float64

In [721]:
price_map = sandbox.dropna(subset=['Item_Number', 'Price_Per_Unit']).set_index('Item_Number')['Price_Per_Unit'].to_dict()
price_map

{10.0: 18.5,
 17.0: 29.0,
 12.0: 21.5,
 16.0: 27.5,
 6.0: 12.5,
 1.0: 5.0,
 22.0: 36.5,
 3.0: 8.0,
 2.0: 6.5,
 24.0: 39.5,
 13.0: 23.0,
 7.0: 14.0,
 4.0: 9.5,
 14.0: 24.5,
 20.0: 33.5,
 25.0: 41.0,
 11.0: 20.0,
 21.0: 35.0,
 23.0: 38.0,
 8.0: 15.5,
 5.0: 11.0,
 19.0: 32.0,
 15.0: 26.0,
 18.0: 30.5,
 9.0: 17.0}

I am imputing the missing (NaN) values utilizing the predefined mapping dictionary.

In [722]:
sandbox['Item_Number'] = sandbox['Item'].str.split('_').str[1].astype(float)
price_map = sandbox.dropna(subset=['Item_Number', 'Price_Per_Unit']).set_index('Item_Number')['Price_Per_Unit'].to_dict()

sandbox['Price_Per_Unit'] = np.where(
    sandbox['Price_Per_Unit'].isna() & sandbox['Item_Number'].notna(), 
    sandbox['Item_Number'].map(price_map), 
    sandbox['Price_Per_Unit']
)

In [723]:
sandbox.isna().sum()

Transaction_Id         0
Customer_Id            0
Category               0
Item                1213
Price_Per_Unit         0
Quantity             604
Total_Spent          604
Payment_Method         0
Location               0
Transaction_Date       0
Discount_Applied    4199
Item_Number         1213
dtype: int64

# Analysis of the Item column #

In [724]:
sandbox['Item'].sort_values().unique()

array(['item_10_bev', 'item_10_but', 'item_10_cea', 'item_10_ehe',
       'item_10_food', 'item_10_fur', 'item_10_milk', 'item_10_pat',
       'item_11_bev', 'item_11_but', 'item_11_cea', 'item_11_ehe',
       'item_11_food', 'item_11_fur', 'item_11_milk', 'item_11_pat',
       'item_12_bev', 'item_12_but', 'item_12_cea', 'item_12_ehe',
       'item_12_food', 'item_12_fur', 'item_12_milk', 'item_12_pat',
       'item_13_bev', 'item_13_but', 'item_13_cea', 'item_13_ehe',
       'item_13_food', 'item_13_fur', 'item_13_milk', 'item_13_pat',
       'item_14_bev', 'item_14_but', 'item_14_cea', 'item_14_ehe',
       'item_14_food', 'item_14_fur', 'item_14_milk', 'item_14_pat',
       'item_15_bev', 'item_15_but', 'item_15_cea', 'item_15_ehe',
       'item_15_food', 'item_15_fur', 'item_15_milk', 'item_15_pat',
       'item_16_bev', 'item_16_but', 'item_16_cea', 'item_16_ehe',
       'item_16_food', 'item_16_fur', 'item_16_milk', 'item_16_pat',
       'item_17_bev', 'item_17_but', 'item_17_ce

In [725]:
sandbox['Item'].value_counts()

Item
item_2_bev      126
item_25_fur     113
item_11_fur     110
item_16_milk    109
item_1_milk     109
               ... 
item_5_bev        7
item_13_bev       7
item_13_fur       7
item_21_pat       6
item_3_ehe        5
Name: count, Length: 200, dtype: int64

Creates a dictionary (item_map) to uniquely look up the 'Item' name and using the combination of 'Price_Per_Unit' and 'Category' as the key.

In [726]:
item_map = sandbox.dropna(subset=['Item', 'Price_Per_Unit', 'Category']).set_index(['Price_Per_Unit', 'Category'])['Item'].to_dict()
item_map

{(18.5, 'patisserie'): 'item_10_pat',
 (29.0, 'milk_products'): 'item_17_milk',
 (21.5, 'butchers'): 'item_12_but',
 (27.5, 'beverages'): 'item_16_bev',
 (12.5, 'food'): 'item_6_food',
 (5.0, 'food'): 'item_1_food',
 (27.5, 'furniture'): 'item_16_fur',
 (36.5, 'butchers'): 'item_22_but',
 (8.0, 'butchers'): 'item_3_but',
 (6.5, 'food'): 'item_2_food',
 (39.5, 'patisserie'): 'item_24_pat',
 (27.5, 'milk_products'): 'item_16_milk',
 (29.0, 'patisserie'): 'item_17_pat',
 (23.0, 'electric_household_essentials'): 'item_13_ehe',
 (14.0, 'beverages'): 'item_7_bev',
 (9.5, 'electric_household_essentials'): 'item_4_ehe',
 (18.5, 'food'): 'item_10_food',
 (24.5, 'furniture'): 'item_14_fur',
 (33.5, 'butchers'): 'item_20_but',
 (41.0, 'furniture'): 'item_25_fur',
 (24.5, 'food'): 'item_14_food',
 (36.5, 'patisserie'): 'item_22_pat',
 (20.0, 'food'): 'item_11_food',
 (12.5, 'patisserie'): 'item_6_pat',
 (35.0, 'electric_household_essentials'): 'item_21_ehe',
 (41.0, 'beverages'): 'item_25_bev',
 (

Creates a temporary composite key column by pairing the 'Price_Per_Unit'and 'Category' columns together as tuples.

In [727]:
sandbox['Price_Category_Key'] = list(zip(sandbox['Price_Per_Unit'], sandbox['Category']))
sandbox['Price_Category_Key']

0           (18.5, patisserie)
1        (29.0, milk_products)
2             (21.5, butchers)
3            (27.5, beverages)
4                 (12.5, food)
                 ...          
12570       (38.0, patisserie)
12571         (6.5, beverages)
12572         (14.0, butchers)
12573        (14.0, furniture)
12574             (17.0, food)
Name: Price_Category_Key, Length: 12575, dtype: object

In [728]:
sandbox['Item'] = np.where(
    sandbox['Item'].isna() & sandbox['Price_Category_Key'].notna(), 
    sandbox['Price_Category_Key'].map(item_map),
    sandbox['Item']
)

In [729]:
sandbox.isna().sum()

Transaction_Id           0
Customer_Id              0
Category                 0
Item                     0
Price_Per_Unit           0
Quantity               604
Total_Spent            604
Payment_Method           0
Location                 0
Transaction_Date         0
Discount_Applied      4199
Item_Number           1213
Price_Category_Key       0
dtype: int64

# Analysis of the Quantity and Total_Spent columns #

Both fields exhibit nearly identical patterns of missing data, indicating that the same 600 transactions likely account for the gaps in both variables.

In [730]:
sandbox[sandbox['Total_Spent'].isna() & (sandbox['Quantity'].isna())]

,Transaction_Id,Customer_Id,Category,Item,Price_Per_Unit,Quantity,Total_Spent,Payment_Method,Location,Transaction_Date,Discount_Applied,Item_Number,Price_Category_Key
7,txn_1372952,cust_21,furniture,item_20_fur,33.5,NaN,NaN,digital_wallet,in-store,2024-04-02,true,NaN,"(33.5, furniture)"
15,txn_1809665,cust_14,beverages,item_14_bev,24.5,NaN,NaN,credit_card,in-store,2022-05-11,NaN,NaN,"(24.5, beverages)"
19,txn_4206593,cust_01,furniture,item_21_fur,35.0,NaN,NaN,digital_wallet,online,2025-01-13,false,NaN,"(35.0, furniture)"
25,txn_3481599,cust_05,furniture,item_24_fur,39.5,NaN,NaN,cash,online,2022-09-08,false,NaN,"(39.5, furniture)"
34,txn_1621497,cust_06,patisserie,item_13_pat,23.0,NaN,NaN,cash,in-store,2023-02-18,NaN,NaN,"(23.0, patisserie)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12527,txn_1069238,cust_23,food,item_1_food,5.0,NaN,NaN,digital_wallet,in-store,2022-08-13,false,NaN,"(5.0, food)"
12552,txn_4823896,cust_05,milk_products,item_3_milk,8.0,NaN,NaN,cash,in-store,2022-07-21,false,NaN,"(8.0, milk_products)"
12556,txn_4397672,cust_04,beverages,item_25_bev,41.0,NaN,NaN,credit_card,online,2024-11-28,true,NaN,"(41.0, beverages)"
12562,txn_7422454,cust_07,butchers,item_20_but,33.5,NaN,NaN,cash,online,2023-04-15,NaN,NaN,"(33.5, butchers)"


# Analysis of the Discount_Applied column #

In [731]:
sandbox['Discount_Applied'].value_counts(dropna=False)

Discount_Applied
true     4219
NaN      4199
false    4157
Name: count, dtype: int64

In [732]:
sandbox['Discount_Applied'] = sandbox['Discount_Applied'].apply(
    lambda x: 'yes' if x == 'true' else ('not' if x == 'false' else x)
)

In [733]:
sandbox[sandbox['Discount_Applied'].isna()]

,Transaction_Id,Customer_Id,Category,Item,Price_Per_Unit,Quantity,Total_Spent,Payment_Method,Location,Transaction_Date,Discount_Applied,Item_Number,Price_Category_Key
3,txn_9458126,cust_06,beverages,item_16_bev,27.5,9.0,247.5,credit_card,online,2022-05-07,NaN,16.0,"(27.5, beverages)"
5,txn_7482416,cust_09,patisserie,item_11_pat,20.0,10.0,200.0,credit_card,online,2023-11-30,NaN,NaN,"(20.0, patisserie)"
14,txn_2490363,cust_09,milk_products,item_16_milk,27.5,2.0,55.0,digital_wallet,online,2022-05-22,NaN,16.0,"(27.5, milk_products)"
15,txn_1809665,cust_14,beverages,item_14_bev,24.5,NaN,NaN,credit_card,in-store,2022-05-11,NaN,NaN,"(24.5, beverages)"
17,txn_9634894,cust_15,milk_products,item_16_milk,27.5,10.0,275.0,digital_wallet,online,2022-04-17,NaN,NaN,"(27.5, milk_products)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12562,txn_7422454,cust_07,butchers,item_20_but,33.5,NaN,NaN,cash,online,2023-04-15,NaN,NaN,"(33.5, butchers)"
12568,txn_1096134,cust_06,food,item_4_food,9.5,8.0,76.0,digital_wallet,in-store,2022-06-11,NaN,4.0,"(9.5, food)"
12570,txn_9347481,cust_18,patisserie,item_23_pat,38.0,4.0,152.0,credit_card,in-store,2023-09-03,NaN,23.0,"(38.0, patisserie)"
12572,txn_5306010,cust_11,butchers,item_7_but,14.0,10.0,140.0,cash,online,2024-08-24,NaN,7.0,"(14.0, butchers)"


Discount_Applied is contextual rather than essential, so the sales can still be analyzed. We’ll therefore populate that field with unknown values.

In [734]:
sandbox['Discount_Applied'] = sandbox['Discount_Applied'].fillna('unknown')

In [735]:
sandbox['Discount_Applied'].value_counts(dropna=False)

Discount_Applied
yes        4219
unknown    4199
not        4157
Name: count, dtype: int64

In [736]:
sandbox.isna().sum()

Transaction_Id           0
Customer_Id              0
Category                 0
Item                     0
Price_Per_Unit           0
Quantity               604
Total_Spent            604
Payment_Method           0
Location                 0
Transaction_Date         0
Discount_Applied         0
Item_Number           1213
Price_Category_Key       0
dtype: int64

# Analysis of the Transaction_Date column #

In [737]:
sandbox['Day'] = sandbox['Transaction_Date'].dt.day
sandbox['Month'] = sandbox['Transaction_Date'].dt.month
sandbox['Year'] = sandbox['Transaction_Date'].dt.year
sandbox['Quarter'] = sandbox['Transaction_Date'].dt.quarter

In [738]:
sandbox.tail()

,Transaction_Id,Customer_Id,Category,Item,Price_Per_Unit,Quantity,Total_Spent,Payment_Method,Location,Transaction_Date,Discount_Applied,Item_Number,Price_Category_Key,Day,Month,Year,Quarter
12570,txn_9347481,cust_18,patisserie,item_23_pat,38.0,4.0,152.0,credit_card,in-store,2023-09-03,unknown,23.0,"(38.0, patisserie)",3,9,2023,3
12571,txn_4009414,cust_03,beverages,item_2_bev,6.5,9.0,58.5,cash,online,2022-08-12,not,2.0,"(6.5, beverages)",12,8,2022,3
12572,txn_5306010,cust_11,butchers,item_7_but,14.0,10.0,140.0,cash,online,2024-08-24,unknown,7.0,"(14.0, butchers)",24,8,2024,3
12573,txn_5167298,cust_04,furniture,item_7_fur,14.0,6.0,84.0,cash,online,2023-12-30,yes,7.0,"(14.0, furniture)",30,12,2023,4
12574,txn_2407494,cust_23,food,item_9_food,17.0,3.0,51.0,cash,online,2022-08-06,unknown,9.0,"(17.0, food)",6,8,2022,3


# Eliminates temporary columns and rows lacking critical information #

In [739]:
drop_cols = ['Item_Number', 'Price_Category_Key']
sandbox = sandbox.dropna(subset=drop_cols)

In [740]:
final_count = len(sandbox)

In [741]:
percentage_removed = ((initial_count - final_count) / initial_count) * 100
percentage_removed

9.646123260437376